In [ ]:
import ibis

con = ibis.connect("duckdb://penguins.ddb")

ibis.options.interactive = True



# Create table

In [ ]:

con.create_table(
    "penguins", ibis.examples.penguins.fetch().to_pyarrow(), overwrite=True
)

# Load table

In [ ]:
penguins = con.table("penguins")

In [ ]:
penguins_pdf = penguins.execute()

# Filter

In [ ]:
penguins.filter(penguins.species == "Adelie")

In [ ]:
penguins_pdf.loc[
    penguins_pdf.species == "Adelie"
].head()

In [ ]:
penguins.filter((penguins.island == "Torgersen") & (penguins.species == "Adelie"))

In [ ]:
penguins_pdf.loc[
    (penguins_pdf.island == "Torgersen") & (penguins_pdf.species == "Adelie")
].head()

# Select

In [ ]:
penguins.select("species", "island", "year")


In [ ]:
penguins_pdf[["species", "island", "year"]].head()

# Mutate

In [ ]:
penguins.mutate(
    bill_length_cm=penguins.bill_length_mm / 10,
    continent=ibis.literal("Antarctica")
)

In [ ]:
penguins_pdf.assign(
    bill_length_cm=penguins_pdf.bill_length_mm / 10,
    continent="Antarctica"
).head()

# Selectors

In [ ]:
import ibis.selectors as s

penguins.mutate(bill_length_cm=penguins.bill_length_mm / 10).select(
    ~s.matches("bill_length_mm")
    # match every column except `bill_length_mm`
)

In [ ]:
penguins_pdf.assign(
    bill_length_cm=penguins_pdf.bill_length_mm / 10,
)[[col for col in penguins_pdf.columns if col != "bill_length_mm"]].head()

In [ ]:
penguins.select("island", s.numeric())


In [ ]:
penguins_pdf.select_dtypes("number").head()

# Order by

In [ ]:
penguins.order_by(penguins.flipper_length_mm).select(
    "species", "island", "flipper_length_mm"
)

In [ ]:
penguins_pdf.sort_values(["flipper_length_mm"])[["species", "island", "flipper_length_mm"]].head()

# Aggregates

In [ ]:
penguins.flipper_length_mm.mean()


In [ ]:
penguins_pdf.flipper_length_mm.mean()

In [ ]:
penguins.aggregate([penguins.flipper_length_mm.mean(), penguins.bill_depth_mm.max()])

In [ ]:
penguins_pdf.aggregate(
    {"flipper_length_mm": "mean", "bill_depth_mm": "max"}
)

# Group by

In [ ]:
penguins.group_by(["species", "island"]).aggregate(penguins.bill_length_mm.mean().name("mean_bill_length_mm"))

In [ ]:
penguins_pdf.groupby(["species", "island"]).agg(mean_bill_length_mm=("bill_length_mm", "mean"))